# Filters

> 在`elasticsearch_dsl`中，`Filter`是一种用于限制搜索结果的查询类型。

与`Query`不同，`Filter`不会影响搜索结果的得分，而只是用于过滤文档。这使得`Filter`比`Query`更加高效，特别是对于大型数据集和高并发搜索。此外，elasticsearch_dsl还支持多种其他的Filter类型，例如`Term、Terms、Range、Exists、GeoDistance、Bool`等。可以根据实际需求选择不同的Filter类型进行搜索。

以下是一个简单的例子，展示如何使用`Filter`进行搜索：

In [1]:
from elasticsearch_dsl import Search, Q, connections
from elasticsearch_dsl.query import Term, Terms, Range, ScoreFunction

# 连接到 Elasticsearch
connections.create_connection(hosts=['localhost'])

# 创建一个查询对象
search = Search(index='books')

In [2]:
def search_term(search, term, columns):
    """该函数将按照 Query 进行检索并打印出指定字段"""
    s = search.filter(term)
    response = s.execute()

    for hit in response.hits:
        for col in columns:
            print(f"{col}: {hit.to_dict().get(col)}")

## Term

In [3]:
term = Term(category='test')
search_term(search, term, columns=['title', 'price'])

title: Term
price: 5


In [6]:
term = Term(category='小')
search_term(search, term, columns=['title', 'price', 'category'])

title: 西游记
price: 10
category: 小说
title: 红楼梦
price: 20
category: 小说
title: 水浒传
price: 45
category: 小说
title: 三国演义
price: 30
category: 小说


In [8]:
term = Term(price=10)
search_term(search, term, columns=['title', 'price'])

title: 西游记
price: 10
title: 创意者的突破
price: 10


### 分词效果

In [10]:
search = Search(index='books')
term = Term(category='小说')
search_term(search, term, columns=['title', 'price'])

In [11]:
search = Search(index='books_ik')

term = Term(category='小说')
search_term(search, term, columns=['title', 'price'])

title: 西游记
price: 10
title: 红楼梦
price: 20
title: 水浒传
price: 45
title: 三国演义
price: 30


*其他写法，示例：*


```python
        Q("term", category="books")
```

## Terms

In [13]:
terms = Terms(category=['小说', '传记'])
search_term(search, terms, columns=['title', 'price'])

title: 西游记
price: 10
title: 红楼梦
price: 20
title: 水浒传
price: 45
title: 三国演义
price: 30
title: 创意者的突破
price: 10
title: 追风筝的人
price: 20
title: 马拉拉的故事
price: 45
title: 爱因斯坦传
price: 35


*其他写法，示例：*

```python
        Q('terms', tags=['search', 'python'])
```

## Range

In [14]:
term = Range(price={"gte": 20, "lte": 100})
search_term(search, term, columns=['title', 'price'])

title: 红楼梦
price: 20
title: 水浒传
price: 45
title: 三国演义
price: 30
title: 追风筝的人
price: 20
title: 马拉拉的故事
price: 45
title: 爱因斯坦传
price: 35
title: Python
price: 47


*其他写法，示例：*

```python
        Q("range", price={"gte": 20, "lte": 100})
```

## 多个字段

In [ ]:
ranges = Range(price={"gte": 20, "lte": 100})
terms = Terms(category=['小说', '传记'])

query = ranges & terms

search_term(search, query, columns=['title', 'price'])

在这个例子中，我们使用Filter来过滤搜索结果。

我们创建两个`Filter`，

1. 一个用于匹配`category`字段为`["小说", "传记"]`的文档，
2. 另一个用于匹配`price`字段在20到100之间的文档。

最后，我们将`term`对象组合起来，并执行搜索。

## `post_filter`

总体说来，我们可以使用如下的两个方法来过滤搜索的结果：

- 使用带有 `filter` 子句的布尔查询。搜索请求将布尔过滤器应用于搜索命中和聚合。
- 使用搜索 `API` 的 `post_filter` 参数。搜索请求仅将 `post_filters` 应用于搜索命中的结果中。

In [ ]:
q = Q('match', title='python')
ranges = Range(price={"gte": 20, "lte": 100})

s = search.query(q).post_filter(ranges)

res = s.execute()
res.hits

# End

- [Elasticsearch：过滤搜索结果 - filter 及 post_filter](https://juejin.cn/post/7114169318669025316)

-----